# Simulated Annealing for Tetris Optimization

In [3]:
import numpy as np
import math
import os
os.chdir('C:/Users/USER/Desktop/Tetris/learning_tetris/game_code')
from game_simulation import *

In [ ]:
import random
from math import *
import numpy as np
from scipy import stats
import game_simulation



def simulation_SA(N_iteration, step_size, temp):
    learning_curve = []
    
    s0 = np.random.multivariate_normal(np.zeros(21), 100*np.eye(21))
    best_score = game_simulation.simulation(s0)


    for i in range(N_iteration):
        neighbour = s0 + np.random.multivariate_normal(np.zeros(21), np.eye(21))*step_size
        score_neighbour = game_simulation.simulation(neighbour)
        
        delta = score_neighbour - best_score
        if delta > 0:
            s0, best_score = neighbour, score_neighbour
        else:
            t = temp / float(i+1)
            metropolis = exp(delta/t)
            if np.random.rand() < metropolis:
                s0, best_score = neighbour, score_neighbour
        
        test_scores = []
        for _ in range(10):
            test_scores.append(game_simulation.simulation(s0))

        
        learning_curve.append(np.mean(test_scores))
        print("Iteration {}: Average score : {}".format(i+1, learning_curve[-1]))
    
    return learning_curve


print(simulation_SA(100, 10, 20))  


In [9]:
def SA(cooling, n_iteration, n_samples, rho):

    mean = np.zeros(21)
    cov = 100 * np.eye(21)
    learning_curve = []
    T = 10 # Initial Temperature
    
    for j in range(n_iteration):
        s0 = np.random.multivariate_normal(mean, cov, n_samples)
        s_neighbour = s0 + np.random.multivariate_normal(np.zeros(21), 50*np.eye(21), n_samples)
        scores = [simulation(sample) for sample in s0]
        scores_neighbour = [simulation(sample) for sample in s_neighbour]

        best_indexes = np.argsort(scores)[-int(n_samples * rho):]
        best_samples = s0[best_indexes]
        best_scores = np.array(scores)[best_indexes]

        best_indexes = np.argsort(scores_neighbour)[-int(n_samples * rho):]
        best_neighbours = s_neighbour[best_indexes]
        best_scores_neighbour = np.array(scores_neighbour)[best_indexes]

        theta = np.mean(best_scores) - np.mean(best_scores_neighbour)
        if theta < 0:
            s0 = s_neighbour
            best_samples = best_neighbours
        else:
            p = math.exp(-theta/T)
            u = np.random.random()
            if u < p:
                s0 = s_neighbour
                best_samples = best_neighbours

        mean = np.mean(best_samples, axis = 0)
        cov =  np.cov(best_samples, rowvar = False, bias=True)

    

        test_scores = []
        for _ in range(30):
            test_scores.append(simulation(mean))

        learning_curve.append(np.mean(test_scores))
        print("Iteration {}: Average score : {}".format(j+1, learning_curve[-1]))


        T *= cooling
        
    return(learning_curve)


In [10]:
curve = SA(.5, 50, 50, .1)



Iteration 1: Average score : 0.5666666666666667
Iteration 2: Average score : 3.933333333333333
Iteration 3: Average score : 4.0
Iteration 4: Average score : 7.633333333333334
Iteration 5: Average score : 12.4
Iteration 6: Average score : 11.033333333333333
Iteration 7: Average score : 10.633333333333333


KeyboardInterrupt: 